In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 150

In [22]:
keep_cols = ['node_id', 'sourceID', 'name', 'countries', 'country_codes']
col_order = ['sourceID', 'name', 'countries', 'country_codes']
dtypes = {'node_id':'int32', 'sourceID':'category', 'name':'object', 'country_codes':'category', 'countries':'category'}

bahamas_officer_raw = pd.read_csv('../data/raw/bahamas_leaks/bahamas_leaks.nodes.officer.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
bahamas_officer_raw = bahamas_officer_raw[col_order]

offshore_officer_raw = pd.read_csv('../data/raw/offshore_leaks/offshore_leaks.nodes.officer.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
offshore_officer_raw = offshore_officer_raw[col_order]

panama_officer_raw = pd.read_csv('../data/raw/panama_papers/panama_papers.nodes.officer.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
panama_officer_raw = panama_officer_raw[col_order]

paradise_officer_raw = pd.read_csv('../data/raw/paradise_papers/paradise_papers.nodes.officer.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
paradise_officer_raw = paradise_officer_raw[col_order]

### To Do

- Standardize names (Title Case)
- Find duplicates
- Why are there company names in here??
- Are there names that go across the 4 datasets?

In [27]:
bahamas_officer_raw.to_csv('../data/intermediate/bahamas_officers.csv')
offshore_officer_raw.to_csv('../data/intermediate/offshore_officers.csv')
panama_officer_raw.to_csv('../data/intermediate/panama_officers.csv')
paradise_officer_raw.to_csv('../data/intermediate/paradise_officers.csv')